In [1]:
# ignore non material warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# COMEX Contracts

In [3]:
# COMEX Contracts date matrix

#Gold
dates_matrix_gold = pd.read_excel("./Gold Futures Data.xlsx", sheet_name="Metadata Gold", skiprows=7)

#Silver
dates_matrix_silver = pd.read_excel("./Silver Futures Data.xlsx", sheet_name="Metadata Silver", skiprows=7)

In [4]:
# expiry dates for COMEX are the same of all three contracts for Gold, Silver, and Palladium
last_trade_dates_gold = pd.to_datetime(dates_matrix_gold["Last Trade"])
last_trade_dates_silver = pd.to_datetime(dates_matrix_silver["Last Trade"])

In [5]:
# COMEX Contracts Settlement Prices

# Gold
settlement_prices_gold = pd.read_excel("./Gold Futures Data.xlsx", sheet_name="GOLD PX_SETTLE")

# Silver
settlement_prices_silver = pd.read_excel("./Silver Futures Data.xlsx", sheet_name="SILVER PX_SETTLE")

In [6]:
#set date index 
settlement_prices_gold.set_index("Date",inplace=True)
settlement_prices_silver.set_index("Dates",inplace=True)

## (Almost) Tradable Returns Calculations

It does not recesarily reflects bid-ask spread but it's close enough in my view. We might think how to incorporate transaction costs even better into the returns calculations but I believe this approach will eliminate price jumps.

Basically my understading is (was) that we should combine prices together, but maybe it would be better to work on returns exclusively. 

So, how I see it. We take front contract, assume some number of contracts we enter at the beginning and then adjust it based on the roll


|Time           |Front Contract     |Back Contract| No contracts |
|---------------|-------------------|-------------|--------------|
|T-5            |       90          |  95         | 1000         |
|Roll Day       |       100         |  110        |  100*1000/110|
|T-3            |       101            |  102.2   |909.9|

Obviously, at T+1 Back Contract becomes Front contract, and the trade repeats itself when the expiration of front contract is coming.

Here is what acctualy matters and what is crucial for us! Return calculation :))




### Rolling Returns Calculation

$$
ret(T-3)=\frac{909.9*101-909.9*110}{909.9*110}
$$

And so forth ...

\
(We could be smart about it and whenever roll occur, but it was hard and I gave up. Whenever we roll we use bid-ask spread but then sudennly to calculate daily non-rollable contract we switch back to settlement pricing? Could be cool? I donno...)


Nico! I tried incorportating your calculation into my thinking. Let me know what you think

## Gold

In [7]:
number = 1
no_contracts = 1_000
contracts_table = []
rolled_prices = []
notional_values = []

for i in range(len(settlement_prices_gold)):

    # Record current position
    current_price = settlement_prices_gold.iloc[i, number-1]
    rolled_prices.append(current_price)
    contracts_table.append(no_contracts)
    notional_values.append(no_contracts * current_price)
    
    # 5 days before the last trading day
    if i + 6 < len(settlement_prices_gold) and settlement_prices_gold.index[i+6] == last_trade_dates_gold[number-1]:

        # Adjust contract quantity (price ratio adjustment)
        new_price = settlement_prices_gold.iloc[i, number]
        no_contracts = (current_price * no_contracts) / new_price

        # Move!
        number += 1

In [8]:
results_df_gold = pd.DataFrame({
    "Date": settlement_prices_gold.index[:len(rolled_prices)],
    "Rolled Price": rolled_prices,
    "Number of Contracts": contracts_table,
    "Notional Value": notional_values
})

results_df_gold.set_index("Date", inplace=True)
results_df_gold["Notional Return"] = results_df_gold["Notional Value"].pct_change()
results_df_gold.iloc[1:-1]

,Rolled Price,Number of Contracts,Notional Value,Notional Return
Date,,,,
2000-01-03,288.5,1000.000000,2.885000e+05,0.000000
2000-01-04,282.7,1000.000000,2.827000e+05,-0.020104
2000-01-05,281.1,1000.000000,2.811000e+05,-0.005660
2000-01-06,281.4,1000.000000,2.814000e+05,0.001067
2000-01-07,281.9,1000.000000,2.819000e+05,0.001777
...,...,...,...,...
2025-04-17,3308.7,803.746888,2.659357e+06,-0.005381
2025-04-18,3308.7,803.746888,2.659357e+06,0.000000
2025-04-21,3406.2,803.746888,2.737723e+06,0.029468


## Silver

In [9]:
number = 1
no_contracts = 1_000
contracts_table_silver = []
rolled_prices_silver = []
notional_values_silver = []

for i in range(len(settlement_prices_silver)):

    # Record current position
    current_price = settlement_prices_silver.iloc[i, number - 1]
    rolled_prices_silver.append(current_price)
    contracts_table_silver.append(no_contracts)
    notional_values_silver.append(no_contracts * current_price)
    
    # 5 days before the last trading day
    if i + 6 < len(settlement_prices_silver) and settlement_prices_silver.index[i + 6] == last_trade_dates_silver[number - 1]:

        # Adjust contract quantity (price ratio adjustment)
        new_price = settlement_prices_silver.iloc[i, number]
        no_contracts = (current_price * no_contracts) / new_price

        # Move!
        number += 1

In [10]:
results_df_silver = pd.DataFrame({
    "Date": settlement_prices_silver.index[:len(rolled_prices_silver)],
    "Rolled Price": rolled_prices_silver,
    "Number of Contracts": contracts_table_silver,
    "Notional Value": notional_values_silver
})

results_df_silver.set_index("Date", inplace=True)
results_df_silver["Notional Return"] = results_df_silver["Notional Value"].pct_change()
results_df_silver.iloc[1:-1]

,Rolled Price,Number of Contracts,Notional Value,Notional Return
Date,,,,
2000-01-03,5.413,1000.000000,5413.000000,0.000000
2000-01-04,5.335,1000.000000,5335.000000,-0.014410
2000-01-05,5.170,1000.000000,5170.000000,-0.030928
2000-01-06,5.127,1000.000000,5127.000000,-0.008317
2000-01-07,5.150,1000.000000,5150.000000,0.004486
...,...,...,...,...
2025-04-17,32.420,774.008997,25093.371697,-0.015368
2025-04-18,32.420,774.008997,25093.371697,0.000000
2025-04-21,32.496,774.008997,25152.196381,0.002344


Interestingly enough, dataset does not filter holdays. Returns are 0 at the time. We should deal with it.

If you made it till the end you've earned coffee delivery to your place. Only first place counts!